In [1]:
import os
import numpy as np
import pandas as pd
import json
import cv2

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

from keras import layers, models, optimizers, regularizers
from keras.applications import EfficientNetB0, ResNet50
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# import matplotlib.pyplot as plt

In [2]:
# test if running the GPU version of tensorflow
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
# loads directory of 3 datasets
dir_train = pd.read_csv('dataset/EuroSAT/train.csv')
dir_valid = pd.read_csv('dataset/EuroSAT/validation.csv')
dir_test = pd.read_csv('dataset/EuroSAT/test.csv')


In [4]:
img_dir = pd.concat([dir_train, dir_valid], ignore_index=False)
img_dir = img_dir.iloc[:,1:-1].reset_index().drop(['index'], axis=1)
print(img_dir.shape)
# img_dir = img_dir.iloc[:100, :] # limit sample size when testing
img_dir

(24300, 3)


,Filename,Label,ClassName
0,AnnualCrop/AnnualCrop_142.jpg,0,AnnualCrop
1,HerbaceousVegetation/HerbaceousVegetation_2835...,2,HerbaceousVegetation
2,PermanentCrop/PermanentCrop_1073.jpg,6,PermanentCrop
3,Industrial/Industrial_453.jpg,4,Industrial
4,HerbaceousVegetation/HerbaceousVegetation_1810...,2,HerbaceousVegetation
...,...,...,...
24295,SeaLake/SeaLake_1943.jpg,9,SeaLake
24296,AnnualCrop/AnnualCrop_211.jpg,0,AnnualCrop
24297,Industrial/Industrial_1428.jpg,4,Industrial
24298,AnnualCrop/AnnualCrop_2571.jpg,0,AnnualCrop


In [5]:
# Load images and labels/classes
images = []
classes = []
# labels.typeof()

base_path = 'dataset/EuroSAT/'
for index, row in img_dir.iterrows():
    img_path = os.path.join(base_path, row['Filename'])
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)
    classes.append(row['ClassName'])

# Normalize images
images = np.array(images) / 255.0
labels = pd.get_dummies(classes).values

In [6]:
images.shape

(24300, 224, 224, 3)

In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Set up the data augmentation
train_data_generator = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)
valid_data_generator = ImageDataGenerator()

In [8]:
# MODEL_PATH = "https://tfhub.dev/sayakpaul/convnext_base_21k_1k_224_fe/1"

def get_model(res=224, num_classes=10):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(res, res, 3))
    base_model.trainable = False
    
    model = keras.Sequential(
        [
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )
    return model

model_1 = get_model()
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                2

In [9]:
# Compile the model
model_1.compile(optimizer=optimizers.Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
validation_steps = len(X_test) // batch_size

history = model_1.fit(train_data_generator.flow(X_train, y_train, batch_size=batch_size), validation_data=valid_data_generator.flow(X_test, y_test, batch_size=batch_size), validation_steps=validation_steps, epochs=epochs)

# Approximatley 50 mins/epoch (batch 32, ConvNeXt)

Epoch 1/10
608/608 [==============================] - 202s 314ms/step - loss: 2.4490 - accuracy: 0.2387 - val_loss: 2.5210 - val_accuracy: 0.1852
Epoch 2/10
608/608 [==============================] - 165s 271ms/step - loss: 2.1902 - accuracy: 0.3122 - val_loss: 2.1244 - val_accuracy: 0.3280
Epoch 3/10
608/608 [==============================] - 167s 275ms/step - loss: 2.0353 - accuracy: 0.3557 - val_loss: 2.0757 - val_accuracy: 0.3427
Epoch 4/10
608/608 [==============================] - 210s 345ms/step - loss: 1.9206 - accuracy: 0.3870 - val_loss: 1.9762 - val_accuracy: 0.3193
Epoch 5/10
608/608 [==============================] - 151s 248ms/step - loss: 1.8336 - accuracy: 0.4049 - val_loss: 1.8358 - val_accuracy: 0.4317
Epoch 6/10
608/608 [==============================] - 183s 300ms/step - loss: 1.7611 - accuracy: 0.4210 - val_loss: 1.6896 - val_accuracy: 0.4344
Epoch 7/10
608/608 [==============================] - 197s 323ms/step - loss: 1.7091 - accuracy: 0.4313 - val_loss: 1.6745 -

In [11]:
# Continue training the model for another 10 epochs
additional_epochs = 10
total_epochs = epochs + additional_epochs

history2 = model_1.fit(train_data_generator.flow(X_train, y_train, batch_size=batch_size), validation_dSata=valid_data_generator.flow(X_test, y_test, batch_size=batch_size), validation_steps=validation_steps, epochs=total_epochs, initial_epoch=epochs)


Epoch 11/20
608/608 [==============================] - 179s 293ms/step - loss: 1.5853 - accuracy: 0.4629 - val_loss: 1.5202 - val_accuracy: 0.5517
Epoch 12/20
608/608 [==============================] - 189s 310ms/step - loss: 1.5673 - accuracy: 0.4717 - val_loss: 1.5356 - val_accuracy: 0.4996
Epoch 13/20
608/608 [==============================] - 189s 311ms/step - loss: 1.5423 - accuracy: 0.4805 - val_loss: 1.5088 - val_accuracy: 0.4828
Epoch 14/20
608/608 [==============================] - 202s 332ms/step - loss: 1.5341 - accuracy: 0.4818 - val_loss: 1.5164 - val_accuracy: 0.5182
Epoch 15/20
608/608 [==============================] - 195s 320ms/step - loss: 1.5254 - accuracy: 0.4853 - val_loss: 1.5380 - val_accuracy: 0.4561
Epoch 16/20
608/608 [==============================] - 201s 330ms/step - loss: 1.5019 - accuracy: 0.4888 - val_loss: 1.4502 - val_accuracy: 0.5627
Epoch 17/20
608/608 [==============================] - 201s 331ms/step - loss: 1.4973 - accuracy: 0.4918 - val_loss: 1

In [12]:
# model_1.save("model_resnet50_nFull_epoch20_batch32_lr.00001")

INFO:tensorflow:Assets written to: model_resnet50_nFull_epoch20_batch32_lr.00001\assets


INFO:tensorflow:Assets written to: model_resnet50_nFull_epoch20_batch32_lr.00001\assets


In [ ]:
# ## if want to load later on: 
# from tensorflow.keras.models import load_model
# model_1 = load_model("model_1_nFull_epoch5_batch32")

In [13]:
# Evaluate the model
batch_size = 32  # Choose a smaller batch size according to your GPU memory capacity
y_pred_batches = []

for i in range(0, len(X_test), batch_size):
    batch_pred = model_1.predict(X_test[i:i + batch_size])
    y_pred_batches.append(batch_pred)

y_pred = np.concatenate(y_pred_batches, axis=0)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# y_pred = model_1.predict(X_test)
# y_pred_classes = np.argmax(y_pred, axis=1)
# y_true = np.argmax(y_test, axis=1)

1/1 [==============================] - 1s 773ms/step


In [14]:
print(classification_report(y_true, y_pred_classes))
print(confusion_matrix(y_true, y_pred_classes))

              precision    recall  f1-score   support

           0       0.51      0.78      0.62       551
           1       0.75      0.07      0.12       547
           2       0.00      0.00      0.00       535
           3       0.23      0.75      0.35       459
           4       0.89      0.83      0.86       453
           5       0.46      0.02      0.03       371
           6       0.50      0.04      0.08       456
           7       0.83      0.81      0.82       502
           8       0.00      0.00      0.00       469
           9       0.32      0.95      0.48       517

    accuracy                           0.43      4860
   macro avg       0.45      0.42      0.34      4860
weighted avg       0.45      0.43      0.34      4860

[[431   0   0  73   1   0   0   1   0  45]
 [  0  36   0   5   0   6   0   0   0 500]
 [168   1   0 239  16   0  10  23   0  78]
 [ 57   2   0 344  10   0   8  13   0  25]
 [  2   0   0  44 378   0   1  28   0   0]
 [ 15   3   0  89   0   6 

c:\Users\Geran\anaconda3\envs\tfgpu_landclass\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Geran\anaconda3\envs\tfgpu_landclass\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Geran\anaconda3\envs\tfgpu_landclass\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

##### If you want to save the model: 

In [ ]:
# model_1.save("model_1")

# ## if want to load later on: 
# from tensorflow.keras.models import load_model
# loaded_model = load_model("model_1")

## Draft / Archive (Plz disregard)

In [ ]:
# import matplotlib.pyplot as plt # somehow this just doesnt work

In [ ]:
# testing/debugging
# print(classes[0])
# images[0].shape

In [ ]:
# ## Use this when running for 1st time
# MODEL_PATH = "https://tfhub.dev/sayakpaul/convnext_base_21k_1k_224_fe/1"
# model = hub.load(MODEL_PATH)
# # saved_model_path = 'model/'
# # tf.saved_model.save(model, saved_model_path)

# # ## Use this for subsequent runs when model already loaded locally
# # saved_model_path = 'model/'
# # model = tf.saved_model.load(saved_model_path)